**Computes the share of motorized transport for each h3 cell (level 9)**

**Input**: csv using the following columns:
* `transportation_mode`: used mode of transport for the trip (see table below)
* `start_time`: datetime of begining of trip
* `end_time`: datetime of end of trip
* `user_id`: Id of the traveling user, used to make sure results include more than one user per geographic division
* `trace_gps`: list of (lon, lat) tuples

**Output**: geoJSON file "../static/data/h3_modal_share.geojson" containing h3 cell shapes with the following metadata:
* `percent`: 0% = all trips were by foot or bicyle, 100% = all trips were motorized
* `color`: A color representation of `percent`, from green to red
* `Sum`: Number of trips in cell made using foot or bicycle
* `Count`: Total number of trips in cell

For anonymity, users are counted only once per cell, and there must be at least 4 trips in a given cell for data to be considered

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import h3pandas
from shapely.geometry import Point, Polygon, LineString
import json
import folium

In [ ]:
df_src = pd.read_csv("sources/data_example.csv")

In [ ]:
tr = {
-10 : "NOT_DEFINED",
0 : "UNKNOWN",
1 : "PASSENGER_CAR",
2 : "MOTORCYCLE",
3 : "HEAVY_DUTY_VEHICLE",
4 : "BUS",
5 : "COACH",
6 : "RAIL_TRIP",
7 : "BOAT_TRIP",
8 : "BIKE_TRIP",
9 : "PLANE",
10 : "SKI",
11 : "FOOT",
12 : "IDLE",
13 : "OTHER",
101 : "SCOOTER",
102 : "HIGH_SPEED_TRAIN"
}
df_src['transportation_mode_tr'] = df_src['transportation_mode'].apply(lambda x: tr[x])

In [ ]:
rows = []
for idx, row in df_src.iterrows():
    trace_gps_list = json.loads(row['trace_gps'])
    num_coords = len(trace_gps_list)
    date_range = pd.date_range(start=row['start_time'], end=row['end_time'], periods=num_coords)
    isNMT = 0
    if (row['transportation_mode']==11 or row['transportation_mode']==8):
        isNMT = 1
    for coord, date in zip(trace_gps_list, date_range):
        rows.append({'lon': coord[0], 'lat': coord[1], 'date': date, 'trace_id': idx, 'isNMT':isNMT, 'user_id': row['user_id'], 'transportation_mode': row['transportation_mode']})

new_df = pd.DataFrame(rows)
new_df

In [ ]:
dfh3 = new_df.h3.geo_to_h3(9, lat_col="lat", lng_col="lon", set_index=False)

In [ ]:
df_unique_user = dfh3.drop_duplicates(subset=['h3_09', 'user_id'])
drawgeoframe = df_unique_user[['h3_09', 'isNMT']].groupby(['h3_09']).agg(Count=('isNMT', np.size), Sum=('isNMT', np.sum))
drawgeoframe

In [ ]:
drawgeoframe = drawgeoframe.h3.h3_to_geo_boundary()

In [ ]:
drawgeoframe['percent'] = drawgeoframe['Sum']*100 / drawgeoframe['Count']
drawgeoframe = drawgeoframe[drawgeoframe['Count'] > 3]
drawgeoframe

In [ ]:
import folium
import branca.colormap as cm
colormap = cm.LinearColormap(["red", "yellow", "green"], vmin=0, vmax=100)
drawgeoframe["color"] = drawgeoframe["percent"].apply(lambda x: colormap(x)[:-2])

start_lat = 48.8915079
start_long = 2.3495425
m = folium.Map(location=[start_lat, start_long], zoom_start=13)
folium.TileLayer('openstreetmap').add_to(m)
folium.TileLayer('cartodbdark_matter').add_to(m)
folium.GeoJson(drawgeoframe, style_function=lambda f: {"color": f['properties']['color']}).add_to(m)
folium.LayerControl().add_to(m)
m

In [ ]:
drawgeoframe.to_file("../static/data/h3_modal_share2.geojson", driver="GeoJSON")